# ↔️ Building Dynamic Video Streams with VideoDB: Integrating Custom Data and APIs

<a href="https://colab.research.google.com/github/video-db/videodb-cookbook/blob/main/examples/Programmatic_Streams_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction
Imagine you're watching a captivating keynote session from your favorite conference, and you’re welcomed with a personalized stream just for you.

This tutorial demonstrates how to create dynamic video streams by integrating data from custom databases and external APIs. We'll use a practical example: a recording of a [Config 2023](https://www.youtube.com/watch?v=Nmv8XdFiej0) keynote session. By using VideoDB, we'll show how companies like [Figma](https://www.figma.com/files/recents-and-sharing?fuid=940498258276625180) can personalize the viewing experience for their audience, delivering a richer and more engaging experience.

We'll showcase how to:

* Fetch data from a random user API to represent a hypothetical viewer.
* Integrate this data into a custom VideoDB timeline.
* Create a personalized stream that dynamically displays relevant information alongside the keynote video.

This tutorial is your guide to unlocking the potential of dynamic video streams and transforming your video content with personalized experiences.

## Setup
---


### 📦  Installing packages

In [ ]:
%pip install videodb

### 🔑 API Keys
Before proceeding, ensure access to [VideoDB](https://videodb.io) API key.

> Get your API key from [VideoDB Console](https://console.videodb.io). ( Free for first 50 uploads, **No credit card required** ) 🎉

In [ ]:
import videodb
import os
from getpass import getpass

# Secure way to enter your VideoDB API key
api_key = getpass("Please enter your VideoDB API Key: ")
os.environ["VIDEO_DB_API_KEY"] = api_key

## Steps
---

### 🔗  Step 1: Connect to VideoDB

Establish a session for uploading videos. Import the necessary modules from VideoDB library to access functionalities.

In [2]:
import videodb
from videodb import connect

conn = videodb.connect()
coll = conn.get_collection()

### 🗳️ Step 2: Upload Base Video
Upload and play the video to ensure it's correctly loaded. We’ll be using [this video](https://www.youtube.com/watch?v=Nmv8XdFiej0) for the purpose of this tutorial.

In [3]:
# Upload and play a video from a URL
video = coll.upload(url="https://www.youtube.com/watch?v=Nmv8XdFiej0")
video.play()

# Alternatively, get a video from your VideoDB collection
# video = coll.get_video('VIDEO_ID_HERE')
# video.play()

'https://console.videodb.io/player?url=https://dseetlpshk2tb.cloudfront.net/v3/published/manifests/5035d388-3b20-48e8-8391-ad157d3784b5.m3u8'

### 📥 Step 3: Fetch Data from a Random User API
This code fetches a random user's data (name and picture) from the "randomuser.me" API.  You can adapt this to retrieve data from any relevant API (e.g., product data, news articles) for your use case.

In [10]:
import requests

# Make a request to the Randomizer API
response = requests.get('https://randomuser.me/api/?results=1&nat=us,ca,gb,au')
data = response.json()

# Extract relevant information
first_name = data['results'][0]['name']['first']
medium_picture = data['results'][0]['picture']['medium']

### 🚥 Step 4 (optional): Prepare Data for Integration

No additional data transformation is required in this example since we are using the data directly from the API. However, in more complex scenarios, you may need to format the data to be suitable for VideoDB.


### 🧱 Step 5: Create VideoDB Assets

We create VideoDB assets for the base video, the user's name (text), and their picture (image). The `TextStyle` object allows us to customize the appearance of the text elements.

In [5]:
from videodb import play_stream, TextStyle, MediaType
from videodb.asset import VideoAsset, TextAsset, ImageAsset
from videodb.timeline import Timeline

# Video Asset
video_asset = VideoAsset(asset_id=video.id, start=0)

# Text Asset with First Name
name_asset = TextAsset(
    text=f'Hi {first_name} !',
    duration=4,
    style=TextStyle(
        fontsize=32,
        font="montserrat",
        borderw=1,
        boxcolor="#D2C11D",
        boxborderw="20",
        x="((w-tw)/2)",
        y="(h-th)/4"
    )
)

# Image Asset with Medium Picture
image = coll.upload(url=medium_picture, media_type=MediaType.image)
image_asset = ImageAsset(
    asset_id=image.id,
    width=80,
    height=80,
    x="275",
    y="230",
    duration=4
)

# Fixed Text Asset
cmon_asset = TextAsset(
    text="Here are your favorite moments",
    duration=4,
    style=TextStyle(
        fontsize=24,
        fontcolor="#D2C11D",
        font="montserrat",
        bordercolor="#D2C11D",
        borderw=1,
        boxborderw="20",
        x="((w-tw)/2)",
        y="(h-200)"
    )
)


### ↔️ Step 6: Create the VideoDB Timeline

The VideoDB timeline allows you to arrange and layer your assets to create a dynamic video stream. In this example, we add the name and picture overlays at a specific time (5 seconds) within the base video.

In [6]:
# Create the timeline
timeline = Timeline(conn)

# Add the base video to the timeline
timeline.add_inline(video_asset)

# Add overlays to the timeline
timeline.add_overlay(5, name_asset)
timeline.add_overlay(5, cmon_asset)
timeline.add_overlay(5, image_asset)

### ▶️ Step 7: Generate and Play the Personalized Stream

The `generate_stream()` method creates a streamable URL for your personalized video stream. You can then use `play_stream()`  to preview it in your browser.

In [7]:
from videodb import play_stream

stream_url = timeline.generate_stream()
print(stream_url)
play_stream(stream_url)

https://dseetlpshk2tb.cloudfront.net/v3/published/manifests/52bedad7-2da2-4664-94a6-bb6d781e7471.m3u8


'https://console.videodb.io/player?url=https://dseetlpshk2tb.cloudfront.net/v3/published/manifests/52bedad7-2da2-4664-94a6-bb6d781e7471.m3u8'

## Conclusion
---

This tutorial showcased how to create personalized video streams using VideoDB. By integrating data from external APIs and custom databases, you can enhance your video content, personalize user experiences, and unlock new possibilities for engagement. Explore various data sources, experiment with different integrations, and customize your video streams to suit your specific needs.